# Course Unit: Using an External AI Agent with Your MCP Shell Server
This notebook provides slide-style explanations and executable code cells demonstrating how to connect an external AI agent (e.g., OpenAI Codex-style agent) to your locally running MCP server.

## Slide 1 — Overview
- Goal: Connect an AI agent to your MCP shell server.
- The agent receives natural language, issues MCP tool calls.
- MCP server performs real Linux operations.
- You learn: AI tool routing, socket connections, agents.

## Slide 2 — Architecture
```
User → AI Agent → MCP Tool Call → Your C MCP Server → Linux CLI → Result → Agent → User
```

## Slide 3 — Start Your MCP Server
Run in terminal:
```
./mcp_server
```

## Slide 4 — Example Agent Code

In [ ]:

from openai import OpenAI
import json

client = OpenAI()

tools = [
    {
        "type": "mcp",
        "mcp_server": {
            "transport": "tcp",
            "host": "127.0.0.1",
            "port": 9000
        }
    }
]

def query_agent(prompt):
    resp = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role":"user","content":prompt}],
        tools=tools,
        tool_choice="auto"
    )
    print(resp)

print("Agent example ready. Ensure your MCP server is running.")


## Slide 5 — Assignment
1. Modify the agent to display tool results cleanly.
2. Add support for delete_older_than_days.
3. Log all agent-to-MCP calls.
4. Demonstrate with 3 natural language queries.
### Extra credit:
https://github.com/combinatrix-ai/ask.sh

https://github.com/codingthefuturewithai/ai-shell-command-generator

This repo generates shell commands from NL (natural language) using an LLM (e.g., Claude).
Modify it to connect to your MCP server as the execution backend instead of executing raw shell commands. Make sure your shell's capabilities are used for RAG, just like before.

### Quickstart
1. curl -LsSf https://astral.sh/uv/install.sh | sh
2. uv tool install ai-shell-command-generator
3. ai-shell --teach -p openai -s bash -q "find large files"
```
Shell Command Generator initialized with Openai (GPT-5 Mini)

════════════════════════════════════════════════════════════
📚 TEACHING MODE
════════════════════════════════════════════════════════════

COMMAND:
  find / -xdev -type f -printf '%s\t%p\n' 2>/dev/null | sort -nr | head -n 20 | awk -F'\t' '{printf "%10s\t%s\n", $1, $2}' | numfmt --to=iec --field=1 --suffix=B

BREAKDOWN:
  - find / -xdev -type f -printf '%s\t%p\n' 2>/dev/null
    - find /                : start search at / (change to any path you want, e.g. /home)
    - -xdev                 : do not descend into other filesystems (avoid /proc, other mounts)
    - -type f               : only regular files
    - -printf '%s\t%p\n'    : print file size in bytes (%s), a tab, then the file path (%p), newline
    - 2>/dev/null           : hide permission and other error messages
  - sort -nr
    - sort numerically (-n) in reverse order (-r) so largest sizes come first
  - head -n 20
    - show only the top 20 results (adjust N as needed)
  - awk -F'\t' '{printf "%10s\t%s\n", $1, $2}'
    - use tab as field separator, format output to ensure the size and path remain separated
  - numfmt --to=iec --field=1 --suffix=B
    - convert the first numeric field (bytes) to human-readable IEC units (KiB, MiB, GiB) and append "B"

OS NOTES:
  - This uses GNU find (-printf) and GNU coreutils (numfmt). On Linux distributions with GNU tools this works as-is.
  - BSD find (macOS) does not support -printf. On BSD/macOS use alternatives such as:
    - find / -type f -exec stat -f '%z\t%N' {} \; 2>/dev/null | sort -nr | ...
    - or use gfind (GNU find) if installed via brew.
  - numfmt is GNU coreutils; if missing, you can drop the numfmt step and read raw bytes or pipe through a different formatter (awk/python).
  - Scanning / may be slow and requires permissions to see all files; run as root to include all files but be cautious.
  - Files with newline characters in names are rare but can break line-oriented pipelines; this pipeline assumes filenames do not contain newlines.

SAFER APPROACH:
  - Preview only a subtree and/or use a size threshold to reduce noise and runtime:
    - Example (preview large files >100M under /home):
      find /home -xdev -type f -size +100M -printf '%s\t%p\n' 2>/dev/null | sort -nr | head -n 50 | awk -F'\t' '{printf "%10s\t%s\n", $1, $2}' | numfmt --to=iec --field=1 --suffix=B
  - If you want to avoid sorting whole filesystem (faster): use du on a specific path:
    - du -b -x /path 2>/dev/null | sort -nr | head -n 20 | numfmt --to=iec --field=1 --suffix=B
  - Run the find command without piping to destructive commands; this pipeline is read-only. Always inspect results before deleting or moving files.

WHAT YOU LEARNED:
  ✓ find -printf '%s %p' prints file sizes in bytes alongside paths for easy sorting.
  ✓ sort -nr orders results by numeric size, largest first.
  ✓ numfmt converts raw byte counts into human-readable units (KiB/MiB/GiB) for easier reading.
  ✓ -xdev prevents crossing filesystems (useful to avoid /proc, other mounts).
  ✓ Limit scope (path, -maxdepth, -size) to speed up searches and reduce noise.

════════════════════════════════════════════════════════════
```